In [4]:
import cv2
import numpy as np
import face_recognition as face_rec
import os
import pyttsx3 as textSpeech
from datetime import datetime

# Initialize text-to-speech engine
engine = textSpeech.init()

def resize(img, size):
    """Resize image by a scaling factor."""
    width = int(img.shape[1] * size)
    height = int(img.shape[0] * size)
    dimension = (width, height)
    return cv2.resize(img, dimension, interpolation=cv2.INTER_AREA)

# Ensure attendance file exists
if not os.path.exists('attendance_report.csv'):
    with open('attendance_report.csv', 'w') as f:
        f.write("Name,Time,Date\n")

# Load images and names
path = 'photos'
studentImg = []
studentName = []
if not os.path.exists(path):
    os.makedirs(path)

myList = os.listdir(path)
for cl in myList:
    curimg = cv2.imread(f'{path}/{cl}')
    if curimg is None:
        print(f"[WARNING] Could not read image {cl}")
        continue
    studentImg.append(curimg)
    studentName.append(os.path.splitext(cl)[0])

def findEncoding(images):
    """Generate face encodings for all known images."""
    imgEncodings = []
    for img in images:
        img = resize(img, 0.50)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodes = face_rec.face_encodings(img)
        if encodes:  # Avoid index error
            imgEncodings.append(encodes[0])
        else:
            print("[WARNING] No face detected in one of the images.")
    return imgEncodings

def MarkAttendance(name):
    """Mark attendance if not already marked today."""
    with open('attendance_report.csv', 'r+') as f:
        myDatalist = f.readlines()
        nameList = [line.split(',')[0] for line in myDatalist[1:]]  # Skip header

        today = datetime.now().strftime('%d-%B-%y')
        # Prevent duplicate marking for same day
        already_marked = any((line.split(',')[0] == name and line.split(',')[2].strip() == today) for line in myDatalist[1:])

        if not already_marked:
            now = datetime.now()
            timestr = now.strftime('%I:%M:%S %p')
            f.write(f'{name},{timestr},{today}\n')
            statement = f'Welcome to class {name}'
            engine.say(statement)
            engine.runAndWait()

# Encode all known images
EncodeList = findEncoding(studentImg)
print(f"[INFO] Encoded {len(EncodeList)} faces.")

# Start webcam feed
vid = cv2.VideoCapture(0)

while True:
    success, frame = vid.read()
    if not success:
        break

    # Resize frame for faster processing
    Smaller_frames = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = cv2.cvtColor(Smaller_frames, cv2.COLOR_BGR2RGB)

    # Detect faces and encode
    facesInFrame = face_rec.face_locations(rgb_small_frame)
    encodeFacesInFrame = face_rec.face_encodings(rgb_small_frame, facesInFrame)

    for encodeFace, faceloc in zip(encodeFacesInFrame, facesInFrame):
        matches = face_rec.compare_faces(EncodeList, encodeFace)
        facedis = face_rec.face_distance(EncodeList, encodeFace)
        matchIndex = np.argmin(facedis)

        if matches[matchIndex]:
            name = studentName[matchIndex].upper()
            y1, x2, y2, x1 = faceloc
            # Scale back to original frame size
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.rectangle(frame, (x1, y2 - 25), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            MarkAttendance(name)

    cv2.imshow('video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

[WARNING] Could not read image .idea
[INFO] Encoded 9 faces.
